# Feature Selection for Categorical Features using Iterative Hard Thresholding

Feature selection is a very important task in machine learning tasks. Finding a relevant subset of features improves generalization, robustness to noise, and convergence to targets. A prominent technique in feature selection *Iterative Hard Thresholding (IHT)*. Although IHT is a powerful technique for feature selection, it is not designed for categorical features. In this notebook, I will give a walkthrough on **how to extend IHT to categorical datasets**. For validation, I will compare IHT to other machine learning techniques that also perform feature selections such as LASSO and Random Forest.


# Iterative Hard Thresholding



Consider a dataset $(\mathbf{X_i}, y_i)$ for $i = [n] = 1,2,3 ... n$, where $\mathbf{X_i} \in \mathbb{R}^m$ and $y_i \in \mathbb{R}.$ We seek to find a k-sparse vector $\mathbf{B} \in  \mathbb{R}^m$ with the following objective: 


$$ \min_{\|\mathbf{B}\|_0 = k} \| \mathbf{y} - f(\mathbf{X} ;\mathbf{B}) \|
$$

where $\mathbf{X} \in \mathbb{R}^{n \times m}, \mathbf{y} \in  \mathbb{R}^{n} $ denote the data matrix and the response vector. The $l_0$ norm denotes the number of non-zero elements in $\mathbf{B}$ and $f$ is a given task-specific function. 





Intuitively, learning $\mathbf{B}$ is equivalent to learning a set of $k$ relevant features that best recover the response vector. Thus, by solving this optimization problem, one can identify relevant features. 

IHT assumes there exists a linear relationship between a subset of the measured variable and the response vector , and seeks to retain the top-k values of the entire feature vector after each iteration. Let $\mathbf{B}^0 = \mathbf{0}$, IHT follows the update rule:

$$
\mathbf{B}^{t+1} =  \mathbf{H}_k(\mathbf{B}^{t} - \lambda \bigtriangledown_{\mathbf{B}^t} L(f(\mathbf{X} ;\mathbf{B}),y))
$$

In [13]:
from jax import grad
import jax.numpy as NP
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# DATASET

In [14]:
epoch = 2000
lr = 0.1
s = 50
dataPath ="/home/tayo/Downloads/THY/IHT_data/molecularData.csv"
labelPath= "/home/tayo/Downloads/THY/IHT_data/molecularLabel.csv"

In [15]:
data = np.genfromtxt(dataPath, dtype= str, delimiter = ",")
label= np.genfromtxt(labelPath, dtype = str, delimiter = ",")

classW = class_weight.compute_class_weight('balanced', np.unique(label), label)
uniqueClasses = np.unique(label).tolist()
numClasses = len(uniqueClasses)
classWL = np.array([classW[uniqueClasses.index(i)] for i in label]).reshape(-1,1) #Calculate weight class for each sample
label = LabelEncoder().fit_transform(label)
X_train, X_test, y_train, y_test = train_test_split(
    data, label, test_size=0.3, random_state=42)

In [22]:
OHC = OneHotEncoder().fit(data)
OHCL = OneHotEncoder().fit(label.reshape(-1,1))
classCount = list(map(len, OHC.categories_)) #Number of all possible values for each categorical feature
indices = np.cumsum(classCount) # For indexing between coefficient and categorical features
DIM = indices[-1]
indices = np.insert(indices,0, 0)
X_trainC = OHC.transform(X_train).toarray()
X_testC = OHC.transform(X_test).toarray()
dataC = OHC.transform(data).toarray()
X_current, Y_current, weight =  None, None, None #Placeholder for batch data
num_train = X_train.shape[0]

/home/tayo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/tayo/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/home/tayo/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform el

ValueError: Found unknown categories [0.0, 1.0] in column 0 during transform

In [17]:
def aggregateFeature(gradients, numSplit):
    gradients = gradients.flatten()
    sumList = []
    current = 0
    for i in numSplit:
        sumList.append(np.mean(np.abs(gradients[current: current + i ])))
        current = current + i
    return sumList

def thresholding(coeff):
    copyCof = coeff[:]
    coeff = np.sum(np.abs(np.array(coeff)), axis = 0) 
    sum_coeff = aggregateFeature(coeff, classCount)
    rankingBest = np.argsort(np.abs(sum_coeff)).ravel()[-s:]
    if rankingBest.shape[0] < s:
        print("rankBest less than s features")
    selected = set([i for j in rankingBest for i in range(indices[j], indices[j+1])]) #List of selected coefficient
    coeff = coeff.flatten()
    notSelected = list(set(range(len(coeff))).difference(selected))
    copyCof[:,notSelected] = 0
    return copyCof

In [18]:
def generateBatch():
    #int(num_train / 5)
    index = np.random.choice(num_train, size =int(num_train / 1), replace =  False)
    global X_current
    global Y_current
    global weight
    X_current = X_trainC[index]
    Y_current = y_train[index].reshape(-1,1)
    weight = classWL[index]

In [19]:
def regression(coeff):  # Define the softmax function
  y = NP.exp(NP.dot(X_current, coeff.T))
  s = NP.expand_dims(NP.sum(y, axis = 1), 1)
  y = y / s
  Y_currentT = OHCL.transform(Y_current).toarray()
  label_logprobs = NP.multiply(NP.log(y) , Y_currentT) #+ NP.multiply(NP.log(1 - y) , (1 - Y_currentT))
  label_logprobs = weight * label_logprobs
  return -NP.mean(label_logprobs)

  
def regressionTest(coeff):  # Define a function
  y = np.exp(np.dot(X_testC, coeff.T))
  s = np.expand_dims(np.sum(y, axis = 1), 1)
  y = y / s 
  return y

In [20]:
cof = np.zeros((numClasses, DIM))
grad_regression  = grad(regression)
for i in range(epoch):
    generateBatch()
    trainError =  regression(cof)
    acc = accuracy_score(np.argmax(regressionTest(cof), axis = 1) , y_test)
    if i % 100 == 0:
        print("Training error:", trainError, "Accuracy:",acc )
    gradient = np.array(grad_regression(cof))
    cof = thresholding(cof - lr * gradient)

/home/tayo/anaconda3/lib/python3.6/site-packages/jax/lib/xla_bridge.py:146: UserWarning: No GPU found, falling back to CPU.
  warnings.warn('No GPU found, falling back to CPU.')


Training error: 0.4223118 Accuracy: 0.25914315569487983
Training error: 0.17078292 Accuracy: 0.9456635318704284
Training error: 0.11761189 Accuracy: 0.9498432601880877
Training error: 0.094772264 Accuracy: 0.9498432601880877
Training error: 0.08183824 Accuracy: 0.9540229885057471
Training error: 0.0733718 Accuracy: 0.955067920585162
Training error: 0.067319535 Accuracy: 0.9571577847439916
Training error: 0.06273099 Accuracy: 0.9571577847439916
Training error: 0.059103437 Accuracy: 0.9571577847439916
Training error: 0.056144744 Accuracy: 0.9561128526645768
Training error: 0.05367279 Accuracy: 0.9561128526645768
Training error: 0.05156764 Accuracy: 0.955067920585162
Training error: 0.049746804 Accuracy: 0.955067920585162
Training error: 0.04815152 Accuracy: 0.9561128526645768
Training error: 0.04673871 Accuracy: 0.9561128526645768
Training error: 0.04547589 Accuracy: 0.9561128526645768
Training error: 0.04433818 Accuracy: 0.9571577847439916
Training error: 0.043306064 Accuracy: 0.9592476

# Baseline Performance

In [21]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(dataC, label, random_state=1, test_size = 0.3)
        

clf = linear_model.SGDClassifier(loss ='log', penalty = 'l1')

clf.fit(X_train, y_train)


print(accuracy_score(y_test, clf.predict(X_test)))


clf = RandomForestClassifier()

clf.fit(X_train, y_train)


print(accuracy_score(y_test, clf.predict(X_test)))

0.9373040752351097
0.9247648902821317


/home/tayo/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/tayo/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
